In [1]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import os.path
import pandas as pd
from unidecode import unidecode
import keyring


In [2]:
def send_email(email_recipient,
               email_bcc,
               email_subject,
               email_message,
               attachment_location = ''):
    ''' This function sends an e-mail with attachments, the inputs should be self-explanatory.
        However it is important to set your own paramenters below where the comments indicates!   
    '''
    email_sender = keyring.get_password("mail_user", "francisco")#Set your own e-mail

    msg = MIMEMultipart()
    msg['From'] = email_sender
    msg['To'] = email_recipient
    msg['Bcc'] = email_bcc
    msg['Subject'] = email_subject

    msg.attach(MIMEText(email_message, 'plain'))

    if attachment_location != '':
        filename = os.path.basename(attachment_location)
        attachment = open(attachment_location, "rb")
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition',
                        "attachment; filename= %s" % filename)
        msg.attach(part)

    try:
        server = smtplib.SMTP(keyring.get_password("mail_server", "francisco"), 587)#set your own server
        server.ehlo()
        server.starttls()
        server.login(keyring.get_password("mail_user", "francisco"), keyring.get_password("mail_pass", "francisco"))#login information, user and password
        server.ehlo()
        server.send_message(msg)
        
 #       text = msg.as_string()
  #      server.sendmail(email_sender, email_recipient, text)
        print('email sent')
        server.quit()
    except:
        print("SMPT server connection error")
    return True




In [4]:
#Read our excel file.
df = pd.read_excel('File.xlsx')
df.head()

,Nome,Curso,Carga,Data,Email
0,Francisco Medeiros,Example,5 hours,2121-04-22,francisco.nania@gmail.com


In [8]:
print(keyring.get_password("mail_user", "francisco"))

francisco@ascongraph.com.br


In [9]:
#Excel sheet is in portuguese, but should make sense
for index in df.index:
    content =  '''This is an edited mail body{}{}{}{}'''.format(df['Nome'][index],df['Curso'][index],df['Data'][index],df['Carga'][index])#Add the text for your e-mail and format with your excel sheet
    file = unidecode(df['Nome'][index]) + '.pdf' #Needed this change because our list of pdf did not have utf-8 characters, but our Names needed
    
    send_email(df['Email'][index] , keyring.get_password("mail_user", "francisco") , 'This is an {}'.format(df['Curso'][index]) , content , file)
   

email sent
